In [ ]:
## source: https://towardsdatascience.com/word-embeddings-for-sentiment-analysis-65f42ea5d26e
## source: https://towardsdatascience.com/twitter-topic-modeling-e0e3315b12e2

In [ ]:
!pip install keras.preprocessing.text
!pip install gensim

ERROR: Could not find a version that satisfies the requirement keras.preprocessing.text (from versions: none)
ERROR: No matching distribution found for keras.preprocessing.text
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 18.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.0
    Uninstalling scipy-1.16.0:
      Successfully uninstalled scipy-1.16.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is th

In [ ]:
import os
import urllib.request
import gzip
from gensim.models import KeyedVectors

# הגדרות
url = 'https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.vec.gz'
gz_path = 'cc.id.300.vec.gz'
vec_path = 'cc.id.300.vec'

# הורדה אם לא קיים
if not os.path.exists(gz_path):
    print("⬇️ Downloading Indo FastText embedding...")
    urllib.request.urlretrieve(url, gz_path)
    print("✅ Download complete!")

# חילוץ הקובץ מ-gzip אם צריך
if not os.path.exists(vec_path):
    print("🧩 Extracting .vec file from .gz...")
    with gzip.open(gz_path, 'rb') as f_in:
        with open(vec_path, 'wb') as f_out:
            f_out.write(f_in.read())
    print("✅ Extraction complete!")


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
import pandas as pd
import numpy as np
import re
import collections
import matplotlib.pyplot as plt
import csv
import seaborn as sns

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import keras
from keras import layers, regularizers, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import *
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import RMSprop
from keras.losses import CategoricalCrossentropy
from keras.metrics import Accuracy

import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.utils import resample
from sklearn.metrics import multilabel_confusion_matrix, classification_report, confusion_matrix

In [ ]:
def custom_tokenizer(df, tk):
    """
    Tokenizes tweets using Keras' Tokenizer after stop word removal

    Args:     df, Pandas DataFrame: Contains all tweets and labels
              tk, Keras Tokenizer Object:  Creates tokens via fit_on_texts fn

    Returns:  df, Pandas DataFrame:  Updated DataFrame with tokenized tweets in new column
    """

    tokenizeList = []

    # remove stopwords
    df.tweet = df.tweet.apply(remove_stopwords)

    tweetList = df['tweet'].tolist()
    tk.fit_on_texts(df['tweet'])
    inv_map = {v: k for k, v in tk.word_index.items()}

    for sentence in tweetList:
        tweet = re.split('\s+', sentence)
        processed_seq = tk.texts_to_sequences(tweet)
        tokens = [inv_map[tok] for seq in processed_seq for tok in seq]
        tokenizeList.append(tokens)

    df['tokens'] = tokenizeList

    return df

In [ ]:
def remove_stopwords(input_text):
    """
    Removes stopwords from tweets based on Indonesian stop word list

    Args:     input_text, string:  tweet

    Returns:  string: cleaned tweet
    """

    stopwords_list = stopwords.words('indonesian')

    words = input_text.split()
    clean_words = [word for word in words if word not in stopwords_list]

    return " ".join(clean_words)

In [ ]:
def embedding_to_matrix(token_dict, embeddings, dimensionality, NB_WORDS):
    """
    Converts Pandas df of pre-trained embeddings to a NxN matrix cross-referenced with
    all tokens in corpus.

    Args:     token_dict, dict: dictionary of tokens representing all tweets
              Embeddings, Pandas DataFrame: pre-trained Embeddings
              Dimensionality, int: dimension of embeddings (i.e. 400-D for Word2Vec or 100-D for FT)
              NB_WORDS, int: Parameter indicating the number of words we'll put in the embedding dictionary

    Returns:  emb_matrix, NumPy Array:  NxN matrix cross-referenced with all tokens in corpus
    """

    emb_dict = {}

    for line in embeddings:
        word = line[0]
        vector = np.asarray(line[1:], dtype='float32')
        emb_dict[word] = vector

    emb_matrix = np.zeros((NB_WORDS, dimensionality))

    for w, i in token_dict:
        # The word_index contains a token for all words of the training data so we need to limit that
        if i < NB_WORDS:
            vect = emb_dict.get(w)
            # Check if the word from the training data occurs in the pre-trained word embeddings
            # Otherwise the vector is kept with only zeros
            if vect is not None:
                emb_matrix[i] = vect
        else:
            break

    return emb_matrix

In [ ]:
def dict_of_tokens(df, tk):
    """
    Creates a dictionary of tokens found in the tweets.

    Args:     df, Pandas Dataframe: Contains all tweets and labels
              tk, Keras Tokenizer Object:  Creates dict of tokens via fit_on_texts fn

    Returns:  dictionary of tokens
    """

    tk.fit_on_texts(df['tweet'])

    return tk.word_index.items()

In [ ]:
def convert_text_to_sequences(df, tk):
    """
    Converts tokenized tweets to sequences.

    Args:       df, Pandas Dataframe: Contains all tweets, tokenized tweets and labels
                tk, Keras Tokenizer Object:  Converts tokenized tweets to sequences via texts_to_sequences fn

    Returns:    NumPy array:  List of all tokenized tweets represented as a sequence of numbers
    """

    tokens = df['tokens']

    return tk.texts_to_sequences(tokens)


In [ ]:
def import_embedding(filepath, Dimensionality):
    """
    Imports a pre-trained embedding file and converts to NumPy array.

    Args:       filepath, string:  file path for embedding data
                Dimensionality, int: dimension of embeddings (i.e. 400-D for Word2Vec or 100-D for FT)

    Returns:    NumPy array:  pre-trained embeddings
    """

    header_list = list(range(0, Dimensionality+1))

    embeddings = pd.read_csv(filepath, delimiter=' ', skiprows=1, index_col=False, names=header_list)

    return embeddings.to_numpy()


In [ ]:
def sequence_pad(X_seq):
    """
    Pads the sequences to some length of max tweet in tokens.

    Args:       X_seq, NumPy array:  List of all tokenized tweets represented as a sequence of numbers

    Returns:    X_seq_trunc, NumPy array:  List of all tokenized tweets appended to value of max tweet length
                MAX_LEN, int:   Maximum tweet length in tokens
    """

    lengths = []

    for i in X_seq:
        lengths.append(len(i))

    MAX_LEN = max(lengths)

    X_seq_trunc = pad_sequences(X_seq, maxlen=MAX_LEN)

    return X_seq_trunc, MAX_LEN

In [ ]:
def encode_labels(df):
    """
    One-hot encodes the emotion labels.

    Args:       df, Pandas DataFrame:  Contains all tweets and labels

    Returns:    y_oh, NumPy array:  one-hot encoded emotion labels
    """

    le = LabelEncoder()

    y = le.fit_transform(df['label'])

    return to_categorical(y)

In [ ]:
def deep_model(model, X_train, y_train, X_valid, y_valid, checkpoint_filepath):
    """
    Function to used for individual model exploration.
    Saves file path at epoch with max validation accuracy.

    Args:       model, Keras object: model with the chosen architecture
                X_train, NumPy array: training features
                y_train, NumPy array: training target
                X_valid, NumPy array: validation features
                Y_valid, NumPy array: validation target
                checkpoint_filepath:  file path to save epoch with max validation accuracy

    Returns:    model training history, Keras object: contains the output of the Keras model.fit fn
    """

    model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
                            filepath=checkpoint_filepath,
                            save_weights_only=False,
                            monitor='val_accuracy',
                            mode='max',
                            save_best_only=True)

    model.compile(optimizer='rmsprop'
                  , loss='categorical_crossentropy'
                  , metrics=['accuracy'])

    history = model.fit(X_train
                       , y_train
                       , epochs=NB_START_EPOCHS
                       , batch_size=BATCH_SIZE
                       , validation_data=(X_valid, y_valid)
                       , verbose=2
                       , callbacks=[model_checkpoint_callback]
                       , shuffle=True )
    return history


def eval_metric(history, metric_name):
    '''
    Function to evaluate a trained model on a chosen metric.
    Training and validation metric are plotted in a
    line chart for each epoch.

    Args:     history, Keras object: model training history
              metric_name, string: loss or accuracy

    Returns:  matplotlib chart:  line chart with epochs of x-axis and metric on y-axis
    '''
    metric = history.history[metric_name]
    val_metric = history.history['val_' + metric_name]

    e = range(1, NB_START_EPOCHS + 1)

    plt.plot(e, metric, label='Train ' + metric_name)
    plt.plot(e, val_metric, label='Validation ' + metric_name)
    plt.title('Bi-LSTM ' + metric_name)
    plt.xlabel('Epochs')
    plt.ylabel(metric_name)
    plt.xticks(np.arange(min(e)+1, max(e)+1, 2.0))
    plt.legend()
    plt.show()

In [ ]:
def make_classification_report(X_valid, y_valid, checkpoint_filepath):
    """
    Makes a classification report.

    Args:     X_valid, NumPy array: validation features
              Y_valid, NumPy array: validation target
              checkpoint_filepath:  file path to save epoch with max validation accuracy

    Returns:  classification report
    """

    model.load_weights(checkpoint_filepath)

    label_names = ["anger", "fear", "joy", "love", "sadness"]

    y_prob = model.predict(X_valid)
    prediction_ints = np.zeros_like(y_prob)
    prediction_ints[np.arange(len(y_prob)), y_prob.argmax(1)] = 1
    prediction = np.where(prediction_ints==1)[1]

    return print(classification_report(y_valid, prediction_ints, target_names=label_names, digits=4))

In [ ]:
def make_confusion_matrix(X_valid, y_valid, checkpoint_filepath):
    """
    Makes a confusion matrix.

    Args:     X_valid, NumPy array: validation features
              Y_valid, NumPy array: validation target
              checkpoint_filepath:  file path to save epoch with max validation accuracy

    Returns:  confusion matrix
    """

    model.load_weights(checkpoint_filepath)

    label_names = ["anger", "fear", "joy", "love", "sadness"]

    y_prob = model.predict(X_valid)
    prediction_ints = np.zeros_like(y_prob)
    prediction_ints[np.arange(len(y_prob)), y_prob.argmax(1)] = 1
    prediction = np.where(prediction_ints==1)[1]

    y_cat_valid_emb = np.where(y_valid==1)[1]

    cf_matrix = confusion_matrix(prediction, y_cat_valid_emb)

    cf_matrix_norm = cf_matrix / cf_matrix.astype(float).sum(axis=1, keepdims=True)

    cf_matrix_norm_round = np.around(cf_matrix_norm, decimals=2)

    df_cm = pd.DataFrame(cf_matrix_norm_round, columns=label_names, index=label_names)

    df_cm.index.name = 'Actual'
    df_cm.columns.name = 'Predicted'

    sns.heatmap(df_cm, cmap='Blues', annot=True)
    plt.yticks(rotation=0)
    plt.title("Confusion Matrix")
    plt.show()

# Main

### Constants

In [ ]:
NB_WORDS = 20000  # Parameter indicating the number of words to be put in the dictionary
NB_START_EPOCHS = 20  # Number of epochs used for training
BATCH_SIZE = 64  # Size of the batches used in the mini-batch gradient descent
MAX_LEN = 64  # Maximum number of words in a sequence
WORD2VEC_DIM = 400  # Number of dimensions of the pre-trained word2vec embeddings
FASTEXT_DIM = 400 # Number of dimensions of the pre-trained fasttext embeddings
NUM_FOLDS = 10 # Number of folds in K-Fold Cross Validation experiment

checkpoint_filepath = '/content/checkpoints.keras'
tweets_filepath = '/content/Twitter_Emotion_Dataset.csv'
word2vec_filepath = '/content/Word2Vec_400dim.txt'
fastext_filepath = word2vec_filepath
TRAIN_SIZE = .9
VALID_SIZE = .1

## Data Handling

### File Read-In

In [ ]:
# import tweets
df = pd.read_csv(tweets_filepath)

# import word2vec pre-trained embeddings
w2v_embeddings = import_embedding(word2vec_filepath, WORD2VEC_DIM)

# # import fasttext pre-trained embeddings
# ft_embeddings = import_embedding(fastext_filepath, FASTEXT_DIM)

### Pre-Processing

In [ ]:
# initialize the tokenizer

tk = Tokenizer(num_words=NB_WORDS,
               filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
               lower=True,
               split=" ")

# tokenize tweets and remove stopwords
df = custom_tokenizer(df, tk)

### Label Encoding

In [ ]:
y_oh = encode_labels(df)

### Create Token Dictionary

In [ ]:
token_dict = dict_of_tokens(df, tk)

### Convert data to numerical sequences

In [ ]:
X_seq = convert_text_to_sequences(df, tk)

### Create word sequences of equal length

In [ ]:
X_seq_trunc, MAX_LEN = sequence_pad(X_seq)

## Features

### Embeddings (Word2Vec and FasText)

In [ ]:
w2v_emb_matrix = embedding_to_matrix(token_dict, w2v_embeddings, WORD2VEC_DIM, NB_WORDS)

ft_emb_matrix = embedding_to_matrix(token_dict, ft_embeddings, FASTEXT_DIM, NB_WORDS)

NameError: name 'embedding_to_matrix' is not defined

### Train / Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_seq_trunc, y_oh, test_size=VALID_SIZE, random_state=12)

## Modeling

## Model Examination

In [ ]:
%%time

model = Sequential()
model.add(Embedding(NB_WORDS, FASTEXT_DIM, input_length=MAX_LEN, ))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(512, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
model.add(GlobalMaxPool1D())
model.add(Dense(512, activation="relu"))
model.add(layers.Dropout(rate=0.5))
model.add(Dense(256, activation="relu"))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(5, activation='softmax'))
model.layers[0].trainable = True
model_history = deep_model(model, X_train, y_train, X_test, y_test, checkpoint_filepath)
print(model.summary())

### Model Accuracy Over Epochs

In [ ]:
eval_metric(model_history, 'accuracy')

### Model Loss Over Epochs

In [ ]:
eval_metric(model_history, 'loss')

### Model Classification Report

In [ ]:
make_classification_report(X_test, y_test, checkpoint_filepath)

### Model Confusion Matrix

In [ ]:
make_confusion_matrix(X_test, y_test, checkpoint_filepath)

## K-Fold Experiment



In [ ]:
%%time

kf = KFold(n_splits=NUM_FOLDS)

i=0

for train_index, test_index in kf.split(X_seq_trunc):
    i+=1
    print("FOLD", i)
    X_train, X_test = X_seq_trunc[train_index], X_seq_trunc[test_index]
    y_train, y_test = y_oh[train_index], y_oh[test_index]

    model = Sequential()
    model.add(Embedding(NB_WORDS, FASTEXT_DIM, input_length=MAX_LEN))
    model.add(SpatialDropout1D(0.2))
    model.add(Bidirectional(LSTM(512, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
    #model.add(Bidirectional(LSTM(256, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
    #model.add(LSTM(512, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
    #model.add(GRU(512, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
    model.add(GlobalMaxPool1D())
    model.add(Dense(512, activation="relu"))
    model.add(layers.Dropout(rate=0.5))
    model.add(Dense(256, activation="relu"))
    model.add(layers.Dropout(rate=0.5))
    model.add(layers.Dense(5, activation='softmax'))
    # model.layers[0].set_weights([ft_emb_matrix])
    model.layers[0].trainable = True
    print(model.summary())

    model.compile(loss="categorical_crossentropy",
                optimizer="rmsprop", metrics=["acc"])

    model.fit(X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=NB_START_EPOCHS, batch_size=BATCH_SIZE, verbose=2)



## Bootstrap Experiment

In [ ]:
%%time

bootdf = pd.DataFrame(X_seq_trunc)

bootdf['label'] = df['label']

for i in range(NUM_FOLDS):

    print("FOLD", i)
    train = resample(bootdf, replace=True, n_samples=3960, random_state=i)
    test = bootdf[~bootdf.index.isin(train.index)]

    X_train = train.iloc[:,:-1]
    X_test = test.iloc[:,:-1]

    y_train = train.iloc[:, -1]
    y_test = test.iloc[:, -1]

    le = LabelEncoder()
    y = le.fit_transform(y_train)
    y_train = to_categorical(y)

    y2 = le.fit_transform(y_test)
    y_test = to_categorical(y2)

    model = Sequential()
    model.add(Embedding(NB_WORDS, FASTEXT_DIM, input_length=MAX_LEN))
    model.add(SpatialDropout1D(0.2))
    model.add(Bidirectional(LSTM(512, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
    #model.add(Bidirectional(LSTM(256, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
    #model.add(LSTM(512, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
    #model.add(GRU(512, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
    model.add(GlobalMaxPool1D())
    model.add(Dense(512, activation="relu"))
    model.add(layers.Dropout(rate=0.5))
    model.add(Dense(256, activation="relu"))
    model.add(layers.Dropout(rate=0.5))
    model.add(layers.Dense(5, activation='softmax'))
    model.layers[0].set_weights([ft_emb_matrix])
    model.layers[0].trainable = True
    print(model.summary())

    model.compile(loss="categorical_crossentropy",
                optimizer="rmsprop", metrics=["acc"])

    model.fit(X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=NB_START_EPOCHS, batch_size=BATCH_SIZE, verbose=2)
